<a href="https://www.kaggle.com/code/franciscosuarezb/crosswalk-detection?scriptVersionId=185984352" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [3]:
data_dir = '/kaggle/input/crosswalk-detection-dataset/Data'
images_dir = os.path.join(data_dir, 'images')
labels_dir = os.path.join(data_dir, 'labels')

In [4]:
# Listar archivos de imágenes y labels
image_files = os.listdir(images_dir)
label_files = os.listdir(labels_dir)

print(f"Total imágenes: {len(image_files)}")
print(f"Total labels: {len(label_files)}")


Total imágenes: 1523
Total labels: 1518


In [5]:
# Dividir en entrenamiento y prueba
train_images, test_images = train_test_split(image_files, test_size=0.15, random_state=42)
# Dividir el conjunto de entrenamiento en entrenamiento y validación
train_images, val_images = train_test_split(train_images, test_size=0.20, random_state=42)  # 0.25 * 0.8 = 0.2

print(f"Total entrenamiento: {len(train_images)}")
print(f"Total validación: {len(val_images)}")
print(f"Total prueba: {len(test_images)}")


Total entrenamiento: 1035
Total validación: 259
Total prueba: 229


In [6]:
output_dir = '/kaggle/working/dataset'

# Crear directorios si no existen
os.makedirs(os.path.join(output_dir, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/labels'), exist_ok=True)


In [7]:
print(f'Train_images: {train_images[:3]}')
print(f'Train_images: {test_images[:3]}')
print(f'Train_images: {val_images[:3]}')

Train_images: ['Strisce--504-_jpg.rf.43a11bdc321bbc9b6a961048ce74bbe0.jpg', 'Strisce--84-_jpg.rf.7eae2d3d0599b9d2cbf498c6cbe6243f.jpg', 'Strisce--360-_jpg.rf.9227409c7f73d123e97ca7ab63659a2d.jpg']
Train_images: ['Strisce--338-_jpg.rf.504896b03879b76a3c88044b77c14022.jpg', 'Strisce--162-_jpg.rf.dae43a9d1a31c52968dd89c4974c3490.jpg', 'Strisce--342-_jpg.rf.7c988bcd55d6c4ab72082717f159cfa3.jpg']
Train_images: ['Strisce--258-_jpg.rf.f80257ec4f8882c6d922f01e78a71d55.jpg', 'Strisce--195-_jpg.rf.024f14a7cba5000bec899b903c56a133.jpg', 'Strisce--226-_jpg.rf.7572d0531ef340e7db84174abd8c0026.jpg']


In [8]:
# Crear una nueva lista con la extensión .txt
train_labels = [f.replace('.jpg', '.txt') for f in train_images]
val_labels=[f.replace('.jpg', '.txt') for f in val_images]
test_labels=[f.replace('.jpg', '.txt') for f in test_images]

print(f' Total Train_Labels:{len(train_labels)}')
print(f'Total Val_Labels:{len(val_labels)}')
print(f'Total test_labels:{len(test_labels)}')

 Total Train_Labels:1035
Total Val_Labels:259
Total test_labels:229


In [9]:
print(f'Train_Labels:{train_labels[:3]}')
print(f'Val_Labels:{val_labels[:3]}')
print(f'Test_Labels:{test_labels[:3]}')


Train_Labels:['Strisce--504-_jpg.rf.43a11bdc321bbc9b6a961048ce74bbe0.txt', 'Strisce--84-_jpg.rf.7eae2d3d0599b9d2cbf498c6cbe6243f.txt', 'Strisce--360-_jpg.rf.9227409c7f73d123e97ca7ab63659a2d.txt']
Val_Labels:['Strisce--258-_jpg.rf.f80257ec4f8882c6d922f01e78a71d55.txt', 'Strisce--195-_jpg.rf.024f14a7cba5000bec899b903c56a133.txt', 'Strisce--226-_jpg.rf.7572d0531ef340e7db84174abd8c0026.txt']
Test_Labels:['Strisce--338-_jpg.rf.504896b03879b76a3c88044b77c14022.txt', 'Strisce--162-_jpg.rf.dae43a9d1a31c52968dd89c4974c3490.txt', 'Strisce--342-_jpg.rf.7c988bcd55d6c4ab72082717f159cfa3.txt']


In [10]:
import os
import shutil

def move_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        dst_file = os.path.join(dst_dir, file_name)
        try:
            shutil.copy(src_file, dst_file)
        except FileNotFoundError:
            print(f"Archivo no encontrado: {src_file}. Creando un archivo .txt vacío.")
            if file_name.endswith('.txt'):
                open(dst_file, 'a').close()

# Mover archivos de entrenamiento
move_files(train_images, images_dir, os.path.join(output_dir, 'train/images'))
move_files(train_labels, labels_dir, os.path.join(output_dir, 'train/labels'))

# Mover archivos de validación
move_files(val_images, images_dir, os.path.join(output_dir, 'val/images'))
move_files(val_labels, labels_dir, os.path.join(output_dir, 'val/labels'))

# Mover archivos de prueba
move_files(test_images, images_dir, os.path.join(output_dir, 'test/images'))
move_files(test_labels, labels_dir, os.path.join(output_dir, 'test/labels'))


Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--118-_jpg.rf.54995eb9bae14606e4cddbc834e8ce37(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--105-_jpg.rf.16ab0e51e3a52d6520b0c449d2a8b56b(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--113-_jpg.rf.1fcdddf1cf3d22ab22bf5b31bdd62a1e(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--107-_jpg.rf.17c8ef02eb84b49a728055218dceaa40(1).txt. Creando un archivo .txt vacío.
Archivo no encontrado: /kaggle/input/crosswalk-detection-dataset/Data/labels/Strisce--105-_jpg.rf.e2fce3b37caa1e8452748ffbadde64df(1).txt. Creando un archivo .txt vacío.


In [11]:
for phase in ['train', 'val', 'test']:
    print(f"{phase} images: {len(os.listdir(os.path.join(output_dir, phase, 'images')))}")
    print(f"{phase} labels: {len(os.listdir(os.path.join(output_dir, phase, 'labels')))}")


train images: 1035
train labels: 1035
val images: 259
val labels: 259
test images: 229
test labels: 229


In [12]:
from datetime import datetime
from functools import wraps

import numpy as np
from IPython.lib.display import Audio
from ultralytics import YOLO


# Crear el yaml archivo de conf de yolov8

In [13]:
yaml_content = """
path: /kaggle/working/dataset

train: train/images
val: val/images


nc: 1

names:
     - crosswalk
"""

# Guardar el archivo
with open('/kaggle/working/crosswalk_dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("Archivo .yaml creado con éxito.")


Archivo .yaml creado con éxito.


# Crear una carpeta para guardar el proyecto

In [14]:
os.makedirs(os.path.join('/kaggle/working/', 'training_results'), exist_ok=True)


In [19]:
proyect_results_path="/kaggle/working/training_results"

## Cargar los pesos pre-entrenados

In [18]:
#%%
from ultralytics import YOLO

epochs = 150 
weight = 'yolov8n.pt'
batch = 64 

model = YOLO(weight)
data_training =  "/kaggle/working/crosswalk_dataset.yaml"
project = proyect_results_path


NameError: name 'proyect_results_path' is not defined

In [ ]:
print(type(model))

In [15]:
%pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
 model.train(data=data_training,
             epochs=epochs,
             imgsz=512,
             batch=batch,
             cache=True,
             patience=20,
             project=project,
             device=[0, 1],
             # Esto especifica el uso de las GPUs 0 y 1
            )

#Crear el YAML para ver los resultados del test

In [22]:
yaml_content = """
path: /kaggle/working/dataset

train: /kaggle/working/
val: test/images


nc: 1

names:
     - crosswalk
"""

# Guardar el archivo
with open('/kaggle/working/crosswalk_test.yaml', 'w') as file:
    file.write(yaml_content)

print("Archivo .yaml creado con éxito.")

Archivo .yaml creado con éxito.


In [23]:
from ultralytics import YOLO

weight = '/kaggle/working/training_results/train3/weights/best.pt'
model = YOLO(weight)


In [29]:
data_test = "/kaggle/working/crosswalk_test.yaml"
# Validar el modelo y especificar el directorio de salida
val_project = '/kaggle/working/validation_results'
val_name = 'crosswalk_val'
metrics = model.val(data=data_test, project=val_project, name=val_name)


Ultralytics YOLOv8.2.46 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /kaggle/working/dataset/test/labels.cache... 229 images, 3 backgrounds, 0 corrupt: 100%|██████████| 229/229 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:03<00:00,  4.09it/s]


                   all        229        362      0.814      0.801      0.896      0.783
Speed: 0.8ms preprocess, 2.9ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /kaggle/working/validation_results/crosswalk_val


In [26]:
import shutil
from pathlib import Path
import os

# Ruta donde YOLO guarda los resultados por defecto
default_val_path = Path('runs/val')

# Nuevo directorio donde quieres mover los resultados
new_val_path = Path('/kaggle/working/validation_results')

# Verificar si el directorio de resultados por defecto existe
if default_val_path.exists() and default_val_path.is_dir():
    # Crear el nuevo directorio si no existe
    new_val_path.mkdir(parents=True, exist_ok=True)
    
    # Mover los resultados de validación al nuevo directorio
    shutil.move(str(default_val_path), str(new_val_path))
    print(f"Resultados de la validación movidos a {new_val_path}")
else:
    print(f"El directorio {default_val_path} no existe. Verifica si la validación se ejecutó correctamente.")

El directorio runs/val no existe. Verifica si la validación se ejecutó correctamente.


In [30]:
import os
import zipfile

def create_zip_from_folders(folders, output_zip_path):
    """
    Crea un archivo .zip de las carpetas especificadas.

    Args:
        folders (list): Lista de rutas de carpetas a incluir en el .zip.
        output_zip_path (str): Ruta del archivo .zip de salida.
    """
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder in folders:
            for root, dirs, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, os.path.join(folder, '..'))
                    zipf.write(file_path, arcname)

# Ejemplo de uso:
folders_to_zip = [
    '/kaggle/working/training_results',
    '/kaggle/working/validation_results',
    '/kaggle/working/runs',
    '/kaggle/working/yolov8n.pt'
    
]
output_zip_path = '/kaggle/working/crosswalk_dataset.zip'

create_zip_from_folders(folders_to_zip, output_zip_path)

print(f"Archivo .zip creado en {output_zip_path}")


Archivo .zip creado en /kaggle/working/crosswalk_dataset.zip
